In [1]:
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

from deep_hedging import Frequency, ConstantRateCurve
from deep_hedging.linear.commodity_bond import CommodityBond

## Input parameters.

In [3]:
RF_RATE = 0.15
Z_SPREAD = 0.05
CONVENIENCE_YIELD = 0.02

TIME_TILL_MATURITY = 2.0
FREQUENCY = Frequency.ANNUALLY

## Create curves.

In [4]:
yield_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE + Z_SPREAD,
    compounding_frequency=Frequency.ANNUALLY,
)
forward_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE - CONVENIENCE_YIELD,
    compounding_frequency=Frequency.ANNUALLY,
)

## Create bond.

### Check the composition of the bond.

In [5]:
start = dt.datetime.today()
end = start + dt.timedelta(days=int(round(TIME_TILL_MATURITY * 365)))

gold_bond = CommodityBond(
    yield_curve=yield_curve,
    start_date=start,
    end_date=end,
    frequency=FREQUENCY,
    yield_curve_commodity=forward_curve,
)
gold_bond

StructuredNote of:
1. LONG 1.356 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 2.0 years
* YTM = 20.0%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2026-07-23 15:32:30.223549.

2. LONG 1.0619 units of Forward:
* Term = 2.0 years
* Strike = 127.69%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2026-07-23 15:32:30.223549.

3. LONG 0.07 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 1.0 years
* YTM = 20.0%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2025-07-24 00:00:00.

4. LONG 0.0619 units of Forward:
* Term = 1.0 years
* Strike = 113.0%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2025-07-24 00:00:00.


### Check the payment schedule.
The schedule accounts for business days and holidays.

In [6]:
gold_bond.schedule

[Timestamp('2024-07-24 00:00:00'),
 Timestamp('2025-07-24 00:00:00'),
 Timestamp('2026-07-24 00:00:00')]

### You can amend the schedule, if needed.

In [7]:
new_schedule = gold_bond.schedule
new_schedule[1] = gold_bond.schedule[1] + dt.timedelta(days=5)
gold_bond.substitute_schedule(new_schedule)
gold_bond.schedule

[Timestamp('2024-07-24 00:00:00'),
 Timestamp('2025-07-29 00:00:00'),
 Timestamp('2026-07-24 00:00:00')]

In [8]:
[5_000 * p.size * ins.payoff() for p, ins in gold_bond.instruments]

[6780.167709292442,
 array([-1470.30185504]),
 350.73512973922317,
 array([-40.86927549])]

## Finally, let's check the coupon that the client will receive.

In [9]:
print(f"{gold_bond.fixed_coupon * 100:.4f}% p.a.")

6.1973% p.a.


In [10]:
gold_bond.price()

1.0

In [11]:
for p, i in gold_bond:
    print(p, i.price())

Position(side=<PositionSide.LONG: 1>, size=1.3560335418584883) 0.6944444444444444
Position(side=<PositionSide.LONG: 1>, size=1.0619731708500968) 0.0
Position(side=<PositionSide.LONG: 1>, size=0.07014702594784464) 0.8312546339370701
Position(side=<PositionSide.LONG: 1>, size=0.061973170850096876) 0.0


In [12]:
gold_bond.instruments

[(Position(side=<PositionSide.LONG: 1>, size=1.3560335418584883),
  ZeroCouponBond:
  * CCY = Currency.RUB
  * Term = 2.0 years
  * YTM = 20.0%
  * Start Date = 2024-07-23 15:32:30.223549
  * End Date = 2026-07-23 15:32:30.223549),
 (Position(side=<PositionSide.LONG: 1>, size=1.0619731708500968),
  Forward:
  * Term = 2.0 years
  * Strike = 127.69%
  * Start Date = 2024-07-23 15:32:30.223549
  * End Date = 2026-07-23 15:32:30.223549),
 (Position(side=<PositionSide.LONG: 1>, size=0.07014702594784464),
  ZeroCouponBond:
  * CCY = Currency.RUB
  * Term = 1.01 years
  * YTM = 20.0%
  * Start Date = 2024-07-23 15:32:30.223549
  * End Date = 2025-07-29 00:00:00),
 (Position(side=<PositionSide.LONG: 1>, size=0.061973170850096876),
  Forward:
  * Term = 1.01 years
  * Strike = 113.1893%
  * Start Date = 2024-07-23 15:32:30.223549
  * End Date = 2025-07-29 00:00:00)]

In [13]:
paths = np.linspace(2_000, 10_000, gold_bond.days_till_maturity).reshape(1, -1)
paths.shape

(1, 730)

In [14]:
p_0 = 5_000
scale = p_0
pay = gold_bond.payoff(paths / p_0) * scale
pay[pay > 0]

array([  374.8994286, 10619.7317085])

In [15]:
b = gold_bond * scale
b

StructuredNote of:
1. LONG 6780.1677 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 2.0 years
* YTM = 20.0%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2026-07-23 15:32:30.223549.

2. LONG 5309.8659 units of Forward:
* Term = 2.0 years
* Strike = 127.69%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2026-07-23 15:32:30.223549.

3. LONG 350.7351 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 1.01 years
* YTM = 20.0%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2025-07-29 00:00:00.

4. LONG 309.8659 units of Forward:
* Term = 1.01 years
* Strike = 113.1893%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2025-07-29 00:00:00.


In [16]:
b.price()

5000.0

In [17]:
gold_bond

StructuredNote of:
1. LONG 6780.1677 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 2.0 years
* YTM = 20.0%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2026-07-23 15:32:30.223549.

2. LONG 5309.8659 units of Forward:
* Term = 2.0 years
* Strike = 127.69%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2026-07-23 15:32:30.223549.

3. LONG 350.7351 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 1.01 years
* YTM = 20.0%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2025-07-29 00:00:00.

4. LONG 309.8659 units of Forward:
* Term = 1.01 years
* Strike = 113.1893%
* Start Date = 2024-07-23 15:32:30.223549
* End Date = 2025-07-29 00:00:00.


In [18]:
gold_bond._cumsum_days

array([370, 730])

In [19]:
s = paths[:, gold_bond._cumsum_days - 1].squeeze(0)
s

array([ 6049.38271605, 10000.        ])

In [20]:
s[0] * gold_bond.fixed_coupon, s[1] * (1 + gold_bond.fixed_coupon)

(374.8994285993515, 10619.731708500967)

In [21]:
b = gold_bond
for pos, inst in b:
    print(inst.__class__.__name__, pos.size * inst.payoff(paths / p_0))

ZeroCouponBond 6780.167709292442
Forward [3839.56399921]
ZeroCouponBond 350.73512973922317
Forward [268.99657876]


In [22]:
for final in np.linspace(3_000, 12_000, 6):
    paths = np.linspace(2_000, final, gold_bond.days_till_maturity).reshape(1, -1)
    # print(paths[:, -1], [pv.size * v.payoff(paths / p_0) for pv, v in b], b.payoff(paths / p_0)[:, -1], final * (1 + gold_bond.fixed_coupon))
    print(
        paths[:, -1],
        b.payoff(paths / p_0)[:, -1] - final * (1 + gold_bond.fixed_coupon),
    )

[3000.] [9.09494702e-13]
[4800.] [9.09494702e-13]
[6600.] [0.]
[8400.] [0.]
[10200.] [1.8189894e-12]
[12000.] [0.]


In [23]:
b[0][0].size * b[0][1].payoff(paths / p_0) + b[1][0].size * b[1][1].payoff(paths / p_0)

array([12743.6780502])

In [24]:
final * (1 + gold_bond.fixed_coupon)

12743.67805020116

In [25]:
g = b.payoff(paths / p_0)
g[g > 0]

array([  437.63770032, 12743.6780502 ])

In [26]:
final = 10_200
paths = np.linspace(2_000, final, gold_bond.days_till_maturity).reshape(1, -1)
print(
    paths[:, -1],
    [pv.size * v.payoff(paths / p_0) for pv, v in b],
    b.payoff(paths / p_0)[:, -1],
    final * (1 + gold_bond.fixed_coupon),
)

[10200.] [6780.167709292442, array([4051.95863338]), 350.73512973922317, array([281.39121293])] [10832.12634267] 10832.126342670987


In [27]:
p_0 * b[1][1].strike

6384.499999999998

In [28]:
b[1][1].payoff(paths / p_0) * p_0 * (1 + gold_bond.fixed_coupon)

array([4051.95863338])

In [29]:
b[1][0].size, p_0 * (1 + gold_bond.fixed_coupon)

(5309.865854250484, 5309.865854250484)

In [30]:
b[1][1].payoff(paths / p_0)

array([0.7631])

In [31]:
(paths / p_0)[:, -1] - b[1][1].strike

array([0.7631])

In [32]:
b[1][1].strike

1.2768999999999997

### TODO:
* Effective yield
* Different CCY for Commodity and Bond